# Import libraries

In [1]:
import os
import logging

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = all messages are logged (default), 1 = INFO, 2 = WARNING, 3 = ERROR
logging.getLogger('tensorflow').setLevel(logging.ERROR)
import time  # Import the time module

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from helpers import parse_variables, get_risk_level, map_to_color, simulate_quant_trait
from models import ols_regression, manhattan_linear, gc
from deep_learning_models import abyss, deep_abyss

from scipy.stats import t
from scipy import stats
from scipy.stats import entropy

from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras import Input, Model, layers, regularizers
from tensorflow.keras.layers import Input, Dense


In [2]:
def divi(arr, effectsize):
    return [(1 / (num + 0.001)) * effectsize for num in arr]

def multi(arr, effectsize):
    return [num * effectsize for num in arr]

In [3]:
# Load genotype

In [4]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
if 'k' not in globals():
    k = int(dict['k'])
    
if 'M' not in globals():
    M = float(dict['M'])

if 'snp_noise' not in globals():
    snp_noise = 0.1
HWE = int(dict['HWE'])

nr_humans = int(dict['nr_humans'])
nr_snps = int(dict['nr_snps'])
bottleneck_nr = int(dict['bottleneck_nr'])

if 'tools' not in globals():
    tools = ['no_corr','abyss_counted', 'abyss', 'abyss_corrected', 'PCA','abyss_pca']

"""
if 'scenarios' not in globals():
    scenarios = ['snp_effect',
                 'linear_continuous',
                 'non_linear_continuous',
                 'discrete_global',
                 'discrete_localized',
                 'mix_linear_continuous',
                 'mix_non_linear_continuous',
                 'mix_discrete_global',
                 'mix_discrete_localized']
"""
if 'scenarios' not in globals():
    scenarios = ['discrete_global']
if 'very_rare_threshold_L' not in globals():
    very_rare_threshold_L = float(dict['very_rare_threshold_L'])
if 'very_rare_threshold_H' not in globals():
    very_rare_threshold_H = float(dict['very_rare_threshold_H'])
if 'rare_threshold_L' not in globals():
    rare_threshold_L = float(dict['rare_threshold_L'])
if 'rare_threshold_H' not in globals():
    rare_threshold_H = float(dict['rare_threshold_H'])
if 'common_threshold_L' not in globals():
    common_threshold_L = float(dict['common_threshold_L'])
if 'common_threshold_H' not in globals():
    common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k


very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

if 'to_analyze' not in globals():
    to_analyze = "complete"

if to_analyze == "complete":
    complete = pd.concat([common, rare, very_rare], axis=1)
elif to_analyze == "common":
    complete = common.copy()
    
elif to_analyze == "rare":
    complete = rare.copy()
elif to_analyze == "very_rare":
    complete = very_rare.copy()
    
complete = ((complete*2)-1)


In [5]:
tools

['no_corr', 'abyss_counted', 'abyss_corrected', 'PCA', 'abyss_pca']

In [6]:
# Names of the environmental risks
proto_naming_dict = {
    "no_risk": "no environmental risk",
    "NW_risk": "Smooth linear North-West environmental risk",
    "N_risk" : "Smooth linear North environmental risk",
    "as_big_blob_risk": "Localised big blob risk",
    "center_risk": "Localised big central risk",
    "big_square_risk": "big square risk",
    "square_risk" : "Tiny square risk",
    'hi_square_risk' : "Tiny square risk",
    "hi_gauss_blob_risk": "Global Gaussian Risk",
    "two_square_risk": "Two tiny risks",
    "gauss_blob_risk" : "Gaussian Risk",
    "three_square_risk": "Three localized Risks",
    "mid_mid_square_risk": "Mid square risk",
    "hi_hyperbole_risk": "Hyperbole risk",
    "sine_risk": "Sinusoidal risk"
    
}

In [7]:
naming_dict = {}

In [8]:
if 'snp_effect' in scenarios:
    naming_dict['no_risk'] = proto_naming_dict['no_risk']
else:
    pass
if 'linear_continuous' in scenarios or 'mix_linear_continuous' in scenarios:
    naming_dict['NW_risk'] = proto_naming_dict['NW_risk']
    naming_dict['N_risk'] = proto_naming_dict['N_risk']
else:
    pass

if 'non_linear_continuous' in scenarios or 'mix_non_linear_continuous' in scenarios:
    naming_dict['hi_gauss_blob_risk'] = proto_naming_dict['hi_gauss_blob_risk']
    naming_dict['sine_risk'] = proto_naming_dict['sine_risk']
    naming_dict['hi_hyperbole_risk'] = proto_naming_dict['hi_hyperbole_risk']
    
else:
    pass

if 'discrete_global' in scenarios or 'mix_discrete_global'in scenarios:
    naming_dict['big_square_risk'] = proto_naming_dict['big_square_risk']
    naming_dict['as_big_blob_risk'] = proto_naming_dict['as_big_blob_risk']
    naming_dict['center_risk'] = proto_naming_dict['center_risk']
    
else:
    pass

if 'discrete_localized' in scenarios or 'mix_discrete_localized'in scenarios:
    naming_dict['hi_square_risk'] = proto_naming_dict['hi_square_risk']
    naming_dict['two_square_risk'] = proto_naming_dict['two_square_risk']
    naming_dict['three_square_risk'] = proto_naming_dict['three_square_risk']
else:
    pass
    

In [9]:
risks = list(naming_dict.keys())

In [10]:
# Load population

In [11]:
"""
populations = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/01_population_structure.pkl")
populations['population_number'] = populations['populations'].str.extract('(\d+)').astype(int)
# Calculating X and Y coordinates
populations['x'] = ((populations['population_number'] - 1) % k) + 1
populations['y'] = ((populations['population_number'] - 1) // k) + 1
populations['z'] = 0.5
populations['population'] = populations['population_number']/(k*k)
palette = [map_to_color(x, y, z, populations) for x, y, z in zip(populations['x'], populations['y'], populations['z'])]

# Standardize the vector
std_dev = np.std(populations['x'])
mean = np.mean(populations['x'])
populations['x'] = np.round((populations['x'] - mean) / std_dev,2)

std_dev = np.std(populations['y'])
mean = np.mean(populations['y'])
populations['y'] = np.round((populations['y'] - mean) / std_dev,2)

# Check the grid
df_agg = populations.groupby(['x', 'y']).agg({'population': 'mean'}).reset_index()

# Now, pivot the aggregated DataFrame
grid_df = df_agg.pivot(index='y', columns='x', values='population')


heatmap = sns.heatmap(grid_df, cmap=palette, linewidths=.5, square=True, cbar=False)

# Add a title to the heatmap
plt.title('Population Grid', fontsize=16)
plt.gca().invert_yaxis()  # Sometimes it's necessary to invert the y-axis for correct orientation
#plt.show()
plt.close()
"""

'\npopulations = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/01_population_structure.pkl")\npopulations[\'population_number\'] = populations[\'populations\'].str.extract(\'(\\d+)\').astype(int)\n# Calculating X and Y coordinates\npopulations[\'x\'] = ((populations[\'population_number\'] - 1) % k) + 1\npopulations[\'y\'] = ((populations[\'population_number\'] - 1) // k) + 1\npopulations[\'z\'] = 0.5\npopulations[\'population\'] = populations[\'population_number\']/(k*k)\npalette = [map_to_color(x, y, z, populations) for x, y, z in zip(populations[\'x\'], populations[\'y\'], populations[\'z\'])]\n\n# Standardize the vector\nstd_dev = np.std(populations[\'x\'])\nmean = np.mean(populations[\'x\'])\npopulations[\'x\'] = np.round((populations[\'x\'] - mean) / std_dev,2)\n\nstd_dev = np.std(populations[\'y\'])\nmean = np.mean(populations[\'y\'])\npopulations[\'y\'] = np.round((populations[\'y\'] - mean) / std_dev,2)\n\n# Check the grid\ndf_agg = populations.groupby([\'x\

In [12]:
populations = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/01_population_structure.pkl")

In [13]:
# PCA
if "PCA" in tools:
    nr_complete_PCs = 8
    pc_columns = ['PC{}'.format(i) for i in range(1, nr_complete_PCs+1)]
    PC_complete = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/PCs/complete_genotypes_AF_0_0.5_.pkl")
else:
    pass

In [22]:
if 'abyss' in tools or 'abyss_pca' in tools or 'abyss_corrected' in tools:
    pq_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_2pqs")][0]
    p2_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_p2s")][0]
    q2_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_q2s")][0]
    
    esti_p2 = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{p2_file}")
    esti_twopq = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{pq_file}")
    esti_q2 = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{q2_file}")
    
    time_p2 = float(p2_file.split('pop_')[1].split('seconds')[0])
    time_q2 = float(q2_file.split('pop_')[1].split('seconds')[0])
    time_pq = float(pq_file.split('pop_')[1].split('seconds')[0])
    total_abyss_time = np.round(time_p2+time_q2+time_pq,3)
    
    if 'abyss_corrected' in tools:
        print('ja')
        correction = pd.DataFrame()
        for snp in list(complete.columns):
            dummies = pd.get_dummies(complete[[snp]], columns=[snp], dtype=int)
            dummies['corr'] = dummies[f"{snp}_-1.0"]*esti_q2[snp] + dummies[f"{snp}_1.0"]*esti_p2[snp]
            correction[snp] = complete[snp]/dummies['corr']


ja


In [24]:
complete

snps,CG2_AF_0.3586,CG7_AF_0.3784,CG8_AF_0.2084,CG13_AF_0.2245,CG14_AF_0.2003,CG16_AF_0.2138,CG19_AF_0.4314,CG20_AF_0.2542,CG23_AF_0.4274,CG25_AF_0.2291,...,VRG136_AF_0.0457,VRG137_AF_0.0407,VRG147_AF_0.0414,VRG155_AF_0.0332,VRG160_AF_0.0478,VRG168_AF_0.0443,VRG170_AF_0.0469,VRG181_AF_0.0355,VRG194_AF_0.031,VRG199_AF_0.0496
0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,-1.0,-1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,-1.0,0.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
3,0.0,1.0,1.0,1.0,1.0,-1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,0.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4996,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4997,-1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4998,1.0,-1.0,1.0,-1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [23]:
correction

,CG2_AF_0.3586,CG7_AF_0.3784,CG8_AF_0.2084,CG13_AF_0.2245,CG14_AF_0.2003,CG16_AF_0.2138,CG19_AF_0.4314,CG20_AF_0.2542,CG23_AF_0.4274,CG25_AF_0.2291,...,VRG136_AF_0.0457,VRG137_AF_0.0407,VRG147_AF_0.0414,VRG155_AF_0.0332,VRG160_AF_0.0478,VRG168_AF_0.0443,VRG170_AF_0.0469,VRG181_AF_0.0355,VRG194_AF_0.031,VRG199_AF_0.0496
0,0.772607,NaN,0.626841,0.785663,0.754404,NaN,0.867934,0.763719,NaN,0.736277,...,0.833389,0.928068,0.672196,0.676342,0.732531,0.715245,0.707612,0.810643,0.695827,0.821542
1,-0.837293,-0.737059,0.768920,0.723052,0.671335,NaN,0.929192,0.872508,NaN,0.762014,...,0.782375,0.811384,0.634931,0.668541,0.816516,0.761681,0.615560,0.837674,0.721173,0.839147
2,-0.762185,NaN,0.703899,0.778301,0.713560,-0.937113,0.926221,0.925146,NaN,0.705299,...,0.825795,0.824991,0.674693,0.679964,0.860370,0.888006,0.722923,0.807928,NaN,0.733383
3,NaN,0.713330,0.742669,0.763451,0.808160,-0.717759,0.827762,NaN,0.919626,0.707665,...,0.693699,0.780307,0.664217,0.762397,0.830856,0.808020,0.672024,0.763484,0.777379,0.822394
4,0.832315,NaN,0.724954,0.710158,0.810455,-0.925197,0.784701,0.961218,NaN,0.727926,...,0.748515,0.727073,0.673765,0.692422,0.787693,0.731924,0.751808,0.825232,0.683110,0.877377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,NaN,NaN,0.995098,NaN,0.823848,0.690143,0.976838,0.764139,NaN,0.784745,...,0.846195,0.885863,NaN,0.821486,0.665014,0.740323,0.753365,0.745041,0.705614,0.721329
4996,0.892799,0.976571,0.742362,NaN,0.815921,0.765377,NaN,0.711627,NaN,0.635780,...,0.736353,0.886281,0.765314,NaN,0.684423,0.677112,0.737028,0.802415,0.837540,0.819273
4997,-0.815839,0.715683,0.772340,NaN,0.755214,0.716316,0.854116,0.800826,0.849585,0.872245,...,0.712736,0.947338,0.695594,0.726200,0.692034,0.768243,0.864457,0.798958,0.739036,0.764850
4998,1.005614,-0.689412,0.851767,-0.614255,0.741267,0.705982,NaN,0.726704,NaN,0.757875,...,0.769785,0.816315,0.760100,0.782831,0.691576,0.828315,0.924045,0.750941,0.768146,0.815607


In [ ]:
if 'abyss' in tools or 'abyss_pca' in tools or 'abyss_corrected' in tools:
    pq_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_2pqs")][0]
    p2_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_p2s")][0]
    q2_file = [f for f in os.listdir(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/") if f.startswith("estimated_q2s")][0]
    
    esti_p2 = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{p2_file}")
    esti_twopq = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{pq_file}")
    esti_q2 = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/{q2_file}")
    
    time_p2 = float(p2_file.split('pop_')[1].split('seconds')[0])
    time_q2 = float(q2_file.split('pop_')[1].split('seconds')[0])
    time_pq = float(pq_file.split('pop_')[1].split('seconds')[0])
    total_abyss_time = np.round(time_p2+time_q2+time_pq,3)
    if 'abyss_corrected' in tools:
        correction = pd.DataFrame()
        for snp in list(complete.columns):
            dummies = pd.get_dummies(complete[[snp]], columns=[snp], dtype=int)

            correction['snp'] = complete[snp]
            correction['minaf'] = esti_q2[snp]
            correction['hetaf'] = esti_twopq[snp]
            correction['majaf'] = esti_p2[snp]
            correction[snp] = correction.apply(lambda row: row['minaf'] if row['snp'] == -1 else (row['hetaf'] if row['snp'] == 0 else row['majaf']), axis=1)
        correction = correction.drop(columns=['snp','minaf','hetaf','majaf'])
else:
    pass

if 'abyss_counted' in tools:
    true_p2s = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/02_true_p2_via_true_pop.pkl")
    true_twopqs = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/02_true_twopq_via_true_pop.pkl")
    true_q2s = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/02_true_q2_via_true_pop.pkl")
else:
    pass

if 'abyss_pca' in tools:
    nr_complete_PCs_abyss = 8
    pc_columns_abyss = ['PC{}'.format(i) for i in range(1, nr_complete_PCs_abyss+1)]
    PC_complete = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/PCs/complete_genotypes_AF_0_0.5_.pkl")
    covariate_dictionary_combi = {}
    for snp in list(complete.columns):
        temp = complete[[snp]].copy()
        temp[f"{snp}_cov_p2minq2"] = esti_p2[snp] - esti_q2[snp]
        temp[pc_columns_abyss] = PC_complete[pc_columns_abyss]
        temp = temp.drop(columns=[snp])
        covariate_dictionary_combi[snp] = temp
else:
    pass

In [ ]:
mu= np.zeros(complete.shape[0])
beta = np.zeros(complete.shape[1])

In [ ]:
ratio_effects = []
if 'snp_effect' in scenarios or len([f for f in scenarios if f.split('_')[0]=='mix']) > 0:
    effectsize = 1
    numbers_af = [float(col.split('_AF_')[1]) for col in common.columns if '_AF_' in col]
    snp_names = [col.split('_AF_')[0] for col in common.columns if '_AF_' in col]
    
    beta_values = divi(numbers_af, effectsize)
    data = {'snp': snp_names, 'Beta': beta_values}
    beta_common = pd.DataFrame(data)
    beta_common['maf'] = "common"
    
    numbers_af = [float(col.split('_AF_')[1]) for col in rare.columns if '_AF_' in col]
    snp_names = [col.split('_AF_')[0] for col in rare.columns if '_AF_' in col]
    
    beta_values = divi(numbers_af, effectsize)
    data = {'snp': snp_names, 'Beta': beta_values}
    beta_rare = pd.DataFrame(data)
    beta_rare['maf'] = "rare"
    
    numbers_af = [float(col.split('_AF_')[1]) for col in very_rare.columns if '_AF_' in col]
    snp_names = [col.split('_AF_')[0] for col in very_rare.columns if '_AF_' in col]
    
    beta_values = divi(numbers_af, effectsize)
    data = {'snp': snp_names, 'Beta': beta_values}
    beta_very_rare = pd.DataFrame(data)
    beta_very_rare['maf'] = "very rare"
    
    betas = pd.concat([beta_common, beta_rare, beta_very_rare], ignore_index=True)

    all_snps = list(complete.columns)
    phenos_mono = []
    for snp in all_snps:
        index_snp = snp.split('_')[0]
        beta_value = betas.loc[betas['snp'] == index_snp, 'Beta'].values[0]
        phenos_mono.append(complete[snp] * beta_value)
    
    # Converting phenos_mono list of series to DataFrame directly
    phenos_mono = pd.concat(phenos_mono, axis=1)
    phenos_mono.columns = complete.columns
    
    # Add noise
    n = len(phenos_mono)
    for snp in list(phenos_mono.columns):
        var_effect = np.var(phenos_mono[snp])
        total_variance = var_effect / snp_noise
        var_noise = total_variance - var_effect
        sd_noise = np.sqrt(var_noise)
        # Generate phenotype with noise
        phenos_mono[snp] = phenos_mono[snp] + np.random.normal(0, sd_noise, n)
        std_dev = np.std(phenos_mono[snp])
        mean = np.mean(phenos_mono[snp])
        phenos_mono[snp] = (phenos_mono[snp] - mean) / std_dev

    if len([f for f in scenarios if f.split('_')[0]=='mix']) > 0:
        ratio_effects.append(0.2)
        ratio_effects.append(0.5)
        ratio_effects.append(0.7)
    else:
        pass
    if 'snp_effect' in scenarios:
        ratio_effects.append(1)
else:
    ratio_effects.append(0)
    phenos_mono = pd.DataFrame()
    phenos_mono[complete.columns[0]] = np.zeros(len(complete))

In [ ]:
path_plots = f"data/results/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/plots/"
path_pvals = f"data/results/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/pvals/"

In [ ]:
os.makedirs(path_plots, exist_ok=True)
os.makedirs(path_pvals, exist_ok=True)

In [ ]:
env_noises = [0.1, 0.5]
if 'snp_effect' in scenarios:
    env_noises = [0.1]

In [ ]:
iter_risks = risks

In [ ]:
performances = pd.DataFrame()
computing = pd.DataFrame()

for ratio_effect in ratio_effects:
    ratio_environment = 1-ratio_effect
    for env_noise in env_noises:
        for name_risk in iter_risks:
            risk = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/environmental_risks/risk_{name_risk}.pkl")
            populations[name_risk] = risk[name_risk]
            df_agg = populations.groupby(['x', 'y']).agg({name_risk: 'mean'}).reset_index()
            grid_df = df_agg.pivot(index='y', columns='x', values=name_risk)
            sns.heatmap(grid_df, cmap='rocket_r', linewidths=.5, square=True, cbar=False)
            
            # Add a title to the heatmap
            plt.title(f"{naming_dict[name_risk]}", fontsize=16)
            plt.gca().invert_yaxis()  # Sometimes it's necessary to invert the y-axis for correct orientation
            plt.savefig(f"{path_plots}/envriskmap_{name_risk}.png", dpi=100)
            #plt.show()
            plt.close()
        
            y = np.array(simulate_quant_trait(mu, np.array(complete), beta, np.array(risk[name_risk]), env_noise))
            # Calculate the standard deviation and mean
            std_dev = np.std(y)
            mean = np.mean(y)
            # Standardize
            y = (y - mean) / std_dev
            risk['pheno'] = y
            y_snps = phenos_mono.copy()
            for snp in list(phenos_mono.columns):
                y_snps[snp] = phenos_mono[snp]*ratio_effect + y*ratio_environment
            y = y_snps.copy()
            
            df_pvals = pd.DataFrame()
            df_time = pd.DataFrame()
            df_bests = pd.DataFrame()
            
            # Best correction
            start_time = time.time()
            df_best_corr = manhattan_linear(complete, y, risk[[f"{name_risk}"]])
            end_time = time.time()
            elapsed_time_best_corr = np.round(end_time - start_time,3)
            
            df_pvals[['snp','coefs','AFs']] = df_best_corr[['snp','coefs','AFs']]
            df_pvals["-logP_best_corr"] = df_best_corr['-logPs']
            df_time["best_corr"] = [elapsed_time_best_corr]
            df_bests["-logP_best_corr"] = np.sort(df_best_corr['-logPs'])
            
            n = len(df_bests)
            expected_quantiles = np.arange(1, n + 1) / n
            expected_logP = np.sort(-np.log10(expected_quantiles))
            df_bests['expected_logP'] = expected_logP
            sns.scatterplot(x='expected_logP', y='-logP_best_corr', data=df_bests, color='black', label=f"Best correction", linewidth=0)

            if "no_corr" in tools:
                # No correction
                start_time = time.time()
                df_no_corr = manhattan_linear(complete, y)
                end_time = time.time()
                elapsed_time_no_corr = np.round(end_time - start_time,3)
                df_pvals['-logP_no_corr'] = df_no_corr['-logPs']
                df_time["no_corr"] = [elapsed_time_no_corr]
                df_bests["-logP_no_corr"] = np.sort(df_no_corr['-logPs'])
                sns.scatterplot(x='expected_logP', y='-logP_no_corr', data=df_bests, color='red', label='no covariate', linewidth=0)
                
            else:
                pass
                


            if "x_y" in tools:
                # X and Y axis as covaraites
                start_time = time.time()
                pops = populations[['x','y']]
                df_pops = manhattan_linear(complete, y , pops)
                end_time = time.time()
                elapsed_time_x_y = np.round(end_time - start_time,3)
                df_pvals["-logP_x_y"] = df_pops['-logPs']
                df_time["x_y"] = [elapsed_time_x_y]
                df_bests["-logP_x_y"] = np.sort(df_pops['-logPs'])
                sns.scatterplot(x='expected_logP', y='-logP_x_y', data=df_bests, color='darkblue', label='X,Y axis as covariate', linewidth=0)

            else:
                pass

            if "abyss_counted" in tools:
                # p2 - q2 and 2pq as covariates
                start_time = time.time()
                df_p_q_2pq_covs_via_true_pops = manhattan_linear(complete, y, true_p2s-true_q2s)
                end_time = time.time()
                elapsed_time_true_ps = np.round(end_time - start_time,3)
                df_pvals["-logP_abyss_via_true_pops"] = df_p_q_2pq_covs_via_true_pops['-logPs']
                df_time["abyss_via_true_pops"] = [elapsed_time_true_ps]
                df_bests["-logP_abyss_via_true_pops"] = np.sort(df_p_q_2pq_covs_via_true_pops['-logPs'])
                sns.scatterplot(x='expected_logP', y='-logP_abyss_via_true_pops', data=df_bests, color='orange', label='True MAFs as covs', linewidth=0)
            else:
                pass

            if "PCA" in tools:
                # PCs
                start_time = time.time()
                df_PCs = manhattan_linear(complete, y , PC_complete[pc_columns])
                end_time = time.time()
                elapsed_time_pcs = np.round(end_time - start_time,3)
                df_pvals["-logP_PCs"] = df_PCs['-logPs']
                df_time["PCs"] = [elapsed_time_pcs]
                df_bests["-logP_PCs"] = np.sort(df_PCs['-logPs'])
                sns.scatterplot(x='expected_logP', y='-logP_PCs', data=df_bests, color='pink', label=f"{nr_complete_PCs} PCs as covariate", linewidth=0)
                
            else:
                pass

            if "abyss" in tools:          
                # Abyss
                start_time = time.time()
                df_abyss_p_q_2pq_covs_via_esti_pop = manhattan_linear(complete, y, esti_p2-esti_q2)            
                end_time = time.time()
                elapsed_time_abyss = np.round(end_time - start_time,3) + total_abyss_time
                df_pvals["-logP_abyss_pq2pq_via_esti_pops"] = df_abyss_p_q_2pq_covs_via_esti_pop['-logPs']
                df_time["abyss_pq2pq_via_esti_pops"] = [elapsed_time_abyss]
                df_bests["-logP_abyss_pq2pq_via_esti_pops"] = np.sort(df_abyss_p_q_2pq_covs_via_esti_pop['-logPs'])
                sns.scatterplot(x='expected_logP', y='-logP_abyss_pq2pq_via_esti_pops', data=df_bests, color='yellow', label='Estimates p2, q2, 2pq as covs', linewidth=0)
                
            else:
                pass
            
            if "abyss_corrected" in tools:          
                # Abyss
                X = complete.copy()
                corr = esti_p2-esti_q2
                start_time = time.time()
                df_corrected = manhattan_linear(corr, y)
                for snp in corr.columns:
                    snp_index = snp.split('_AF_')[0]
                    beta_value = df_corrected.loc[df_corrected['snp'] == snp_index, 'coefs'].values[0]
                    corr[snp] = corr[snp]*beta_value
                new_y = y - corr
                df_abyss_p_q_2pq_covs_via_esti_pop_corrected = manhattan_linear(X-(esti_p2-esti_q2), new_y)            
                end_time = time.time()
                elapsed_time_abyss = np.round(end_time - start_time,3) + total_abyss_time
                df_pvals["-logP_abyss_pq2pq_via_esti_pops_corrected"] = df_abyss_p_q_2pq_covs_via_esti_pop_corrected['-logPs']
                df_time["abyss_pq2pq_via_esti_pops_corrected"] = [elapsed_time_abyss]
                df_bests["-logP_abyss_pq2pq_via_esti_pops_corrected"] = np.sort(df_abyss_p_q_2pq_covs_via_esti_pop_corrected['-logPs'])
                sns.scatterplot(x='expected_logP', y='-logP_abyss_pq2pq_via_esti_pops_corrected', data=df_bests, color='blue', label='Estimates p2, q2, 2pq as covs + correction', linewidth=0)
                
            else:
                pass
            if "abyss_pca" in tools:
                # Combines
                start_time = time.time()
                df_abyss_combined = manhattan_linear(complete, y, covariate_dictionary_combi)
                end_time = time.time()
                elapsed_time_combined = np.round(end_time - start_time,3) + total_abyss_time
                df_pvals["-logP_combined"] = df_abyss_combined['-logPs']
                df_time["combined"] = [elapsed_time_combined]
                df_bests["-logP_combined"] = np.sort(df_abyss_combined['-logPs'])
                sns.scatterplot(x='expected_logP', y='-logP_combined', data=df_bests, color='purple', label='Combined Abyss and PCA', linewidth=0)

            df_pvals.to_pickle(f"{path_pvals}/P_vals_risk_{name_risk}_envnoise_{env_noise}_causalnoise_{snp_noise}_ratioeffect_{ratio_effect}.pkl")
            
            # Find the maximum value in the DataFrame excluding inf and NaN
            max_value = df_bests.replace([np.inf, -np.inf], np.nan).max().max()
            
            # Replace inf values with the maximum value found
            df_bests.replace([np.inf, -np.inf], max_value, inplace=True)
            
            # Replace NaN values with the maximum value found
            df_bests.fillna(max_value, inplace=True)
            
            
            
            df_differences = df_bests.subtract(df_bests['-logP_best_corr'], axis=0).abs()
            performances[f"risk_{name_risk}_envnoise_{env_noise}_causalnoise_{snp_noise}_ratioeffect_{ratio_effect}"] = list(df_differences.mean(axis=0))
            computing[f"risk_{name_risk}_envnoise_{env_noise}_causalnoise_{snp_noise}_ratioeffect_{ratio_effect}"] = list(df_time.T[0])
        
            
            
            # Plot diagonal reference line
            plt.plot([min(df_bests['expected_logP']), max(df_bests['expected_logP'])], 
                     [min(df_bests['expected_logP']), max(df_bests['expected_logP'])], 
                     color='red', linestyle='--')
            
            # Set plot labels and title
            plt.xlabel('Expected')
            plt.ylabel('-Log10(P) Values')
            plt.title(f"QQ Plot of Log Values - {naming_dict[name_risk]},\nSNP effect={ratio_effect*100}%, noise={np.round(snp_noise*100)}%\n Environment={np.round(ratio_environment*100)}%, noise={env_noise}%")
            
            # Show legend
            
            plt.legend()
            plt.savefig(f"{path_plots}/qq_risk_{name_risk}_envnoise_{env_noise}_causalnoise_{snp_noise}_ratioeffect_{ratio_effect}.png", dpi=100)
            # Show plot
            plt.show()
            plt.close()
    iter_risks = risks

os.system(f"rm -rf data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}")
performances['tools'] = list(df_bests.columns)
computing['tools'] = list(df_time.columns)

path_results = f"data/results/"
os.makedirs(path_results, exist_ok=True)
performances.to_pickle(f"{path_results}/{to_analyze}_geno_G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}_{snp_noise}_performances.pkl")
computing.to_pickle(f"{path_results}/{to_analyze}_geno_G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}_{snp_noise}_computingtimes.pkl")